In [1]:
# !pip install pandas_datareader

In [2]:
# import math
import numpy as np
# import os
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
# import matplotlib.pyplot as plt
# plt.style.use('ggplot')
import pandas_datareader as pdr
from datetime import datetime, timedelta

In [3]:
#Get the stock
ticker = 'SYY'
stock = "Sysco"
today = datetime.today()
model_file_name = 'Model_SYY.h5'

In [4]:
# for predictions add 100 extra days to make sure it has more than ts_points to make up for missing days (weekends and holidays)
# while calling thos model we will have to pass the excat number of ts_points as that were used while training the model
ts_points = 120
fetch_days = ts_points + 100    

sd =  today - timedelta(days=fetch_days)
sd = sd.strftime('%Y-%m-%d')

ed = today.strftime('%Y-%m-%d')
ed

next_day = (today + timedelta(days=1) ).strftime('%Y-%m-%d')
# next_day

In [5]:
# get the quote
quote = pdr.DataReader(ticker, data_source="yahoo", start=sd, end=ed)
new_df = quote.filter(['Close'])
last_n_days = ts_points
last_n_days_arr = new_df[-last_n_days:].values 

In [6]:
scaler = MinMaxScaler(feature_range=(0,1))
last_n_days_scaled = scaler.fit_transform(last_n_days_arr)
X_test = []
X_test.append(last_n_days_scaled)
X_test = np.array(X_test)
X_test = np.reshape(X_test,(X_test.shape[0], X_test.shape[1],1))


In [7]:
##########################loading saved model ###############################
new_model = tf.keras.models.load_model('Saved_Models/'+model_file_name)
# Check its architecture
new_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 120, 50)           10400     
_________________________________________________________________
lstm_7 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dropout_3 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 25)                1275      
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 26        
Total params: 31,901
Trainable params: 31,901
Non-trainable params: 0
_________________________________________________________________


In [8]:
# Use The Model To Predict
nmpred_price = new_model.predict(X_test)
nmpred_price = scaler.inverse_transform(nmpred_price)
nmpred_price = nmpred_price[0][0]
print(f"Predicted Closing price for '{stock}' on {next_day} is {nmpred_price:.4f}")

Predicted Closing price for 'Sysco' on 2021-10-25 is 80.8408


In [9]:
# Look at last 4 days to see if it is close 
d1 = today - timedelta(days=5)
d1 = d1.strftime('%Y-%m-%d')

quote2 = pdr.DataReader(ticker, data_source="yahoo", start=d1)
quote2

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2021-10-19,81.730003,79.489998,81.680000,81.339996,2066400,81.339996
2021-10-20,81.580002,79.680000,81.339996,79.820000,2737600,79.820000
2021-10-21,80.760002,79.330002,79.900002,79.629997,1567100,79.629997
2021-10-22,79.989998,78.639999,79.540001,78.680000,1418400,78.680000
